In [3]:
import pandas as pd
import geopandas as gpd
from siuba import *

In [4]:
import zipfile

In [5]:
# ! pip install pygris

In [6]:
import _utils
import importlib
importlib.reload(_utils)

<module '_utils' from '/home/jovyan/data-analyses/sb125_analyses/vmt_transit_sketch/_utils.py'>

In [10]:
import shared_utils
from calitp_data_analysis import geography_utils

# Selecting Corridors

In [8]:
analysis_date = '2023-04-15'

In [ ]:
feeds = shared_utils.gtfs_utils_v2.schedule_daily_feed_to_gtfs_dataset_name(selected_date=analysis_date)

In [9]:
tracts = _utils.get_tract_geoms()

Using FIPS code '06' for input 'CA'


In [ ]:
def trips_to_stops(trip_df, feed_list):
    st = shared_utils.gtfs_utils_v2.get_stop_times(analysis_date, feed_list, trip_df=trip_df)
    st = st >> distinct(_.stop_id, _.stop_sequence) >> collect()
    st = stops >> select(_.stop_id, _.geometry) >> inner_join(_, st, on='stop_id')
    return st

In [ ]:
def sjoin_tracts(stops_gdf, tracts_gdf, buffer_m):
    
    stops_gdf = stops_gdf.to_crs(geography_utils.CA_NAD83Albers)
    assert stops_gdf.crs == tracts_gdf.crs
    
    stops_gdf.geometry = stops_gdf.buffer(buffer_m)
    tracts_sjoined = gpd.sjoin(tracts_gdf, stops_gdf) >> distinct(_.GEOID, _keep_all=True)
    
    return tracts_sjoined

## MST (table)

In [ ]:
mst = feeds >> filter(_.name.str.contains('Monterey'))
mst

In [ ]:
stops = shared_utils.gtfs_utils_v2.get_stops(analysis_date, mst.feed_key)

In [ ]:
stops.columns

In [ ]:
trips = shared_utils.gtfs_utils_v2.get_trips(analysis_date, mst.feed_key)

In [ ]:
trips.columns

In [ ]:
trips.route_short_name.unique()

In [ ]:
ab_trips = trips >> filter(_.route_short_name.isin(['A', 'B']), _.direction_id == 0)

In [ ]:
trips_20 = trips >> filter(_.route_short_name == '20', _.direction_id == 0)

In [ ]:
st_20 = shared_utils.gtfs_utils_v2.get_stop_times(analysis_date, mst.feed_key, trip_df=trips_20)

In [ ]:
st_20 = st_20 >> distinct(_.stop_id, _.stop_sequence) >> collect()

In [ ]:
st_20 = stops >> select(_.stop_id, _.geometry) >> inner_join(_, st_20, on='stop_id')

In [ ]:
st_20 = trips_to_stops(trips_20)

In [ ]:
#  SURF BRT area for joins...
st_20 = st_20 >> filter(_.stop_sequence <= 27)

In [ ]:
# st_20.explore()

In [ ]:
st_ab = trips_to_stops(ab_trips)

In [ ]:
# st_ab.explore()

In [ ]:
surf_corridor = pd.concat([st_20, st_ab])

In [ ]:
surf_corridor.explore()

## Wilshire

In [ ]:
metro = feeds >> filter(_.name.str.contains('LA Metro Bus'))
metro

In [ ]:
stops = shared_utils.gtfs_utils_v2.get_stops(analysis_date, metro.feed_key)

In [ ]:
trips = shared_utils.gtfs_utils_v2.get_trips(analysis_date, metro.feed_key)

In [ ]:
trips.route_short_name.unique()

In [ ]:
trips_720 = trips >> filter(_.route_short_name.isin(['720']), _.direction_id == 0)

In [ ]:
st_720 = trips_to_stops(trips_720, metro.feed_key)

In [ ]:
#  only keep W of Wil/Wstn
st_720 = st_720 >> filter(_.stop_sequence <= 11)

In [ ]:
st_720.explore()

## Fresno Route 1

## San Pablo Ave

## Eureka Route TBD

## Sjoin and calculate

In [ ]:
surf_corridor = surf_corridor.to_crs(geography_utils.CA_NAD83Albers)
st_720 = st_720.to_crs(geography_utils.CA_NAD83Albers)

In [ ]:
#  half-mile buffer stop areas, corridors should be broader...
surf_corridor.geometry = surf_corridor.buffer(804) 
st_720.geometry = st_720.buffer(804)

In [ ]:
gdf = gdf >> select(-_.index_left, -_.index_right)

In [ ]:
surf = gpd.sjoin(gdf, surf_corridor) >> distinct(_.GEOID, _keep_all=True)

In [ ]:
surf.explore(column = 'projected_new_transit_trips', scheme = 'NaturalBreaks')

In [ ]:
wilshire = gpd.sjoin(gdf, st_720) >> distinct(_.GEOID, _keep_all=True)

In [ ]:
wilshire.explore(column = 'projected_new_transit_trips', scheme = 'NaturalBreaks')

In [ ]:
surf.describe()

In [ ]:
wilshire.describe()

In [ ]:
surf.sum()

In [ ]:
wilshire.sum()